In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import pandas as pd
from scipy.stats import gamma
from scipy.optimize import minimize
import warnings
from create_data import generate_data_true, generate_data_more_var, generate_data_ar2,cumulative_gamma
import os

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import pandas as pd
from scipy.stats import gamma
from scipy.optimize import minimize
import warnings
from create_data import generate_data_true, generate_data_more_var, generate_data_ar2,cumulative_gamma
import os


#set seed for reproducibility
np.random.seed(42)
# Define the common parameters
T = 120
NoCov = 2
R_0 = 3
I_0 = 500
bias_corr_const = np.exp(-0.001 / 2)
Omega = np.array([cumulative_gamma(i, 2.5, 3) - cumulative_gamma(i-1, 2.5, 3)
                  for i in range(1, 26)]) / cumulative_gamma(25, 2.5, 3)
OraclePhi = np.array([0.5, 0.7])
OracleBeta = np.array([-0.02, -0.125]).reshape(NoCov, 1)
OracleBeta1 = np.array([-0.02, -0.125, -0.03]).reshape(NoCov + 1, 1)
OraclePhi1 = np.array([0.5, 0.5, 0.3])
Rmin1 = 2


# Generate data
df0 = generate_data_true(T, NoCov, R_0, I_0, Omega, OraclePhi, OracleBeta, bias_corr_const)
df = generate_data_true(T, NoCov, R_0, I_0, Omega, OraclePhi, OracleBeta, bias_corr_const)
df1 = generate_data_more_var(T, NoCov, R_0, I_0, Omega, OraclePhi, OracleBeta1, bias_corr_const)
df2 = generate_data_ar2(T, NoCov, R_0, I_0, Omega, OraclePhi1, OracleBeta, bias_corr_const, Rmin1)
#define tau
tau_0=5


# Helper Functions
def logistic(x):
    return np.log(x / (1 - x))

def greater_or_na(obj, const, tocat):
    if np.isnan(obj):
        print(f"{obj} {tocat}")
    return np.isnan(obj) or obj > const

def omega_vector(length=25, shape=2.5, scale=3):
    omega = np.array([(gamma.cdf(i, shape, scale=scale) - gamma.cdf(i - 1, shape, scale=scale))
                      / gamma.cdf(length, shape, scale=scale) for i in range(1, length + 1)])
    return omega
Z = df[['Z1', 'Z2']].values
I = df['I'].values

def QSOEID(Z, I, NoCov, T, I_0, R_0, bias_corr_const,tau_0):
    Omega = omega_vector()
    Lambda = np.full(T, np.nan)
    Lambda[0] = I_0 * Omega[0]
    #range 1-24
    for t in range(1, 25):
        Lambda[t] = I_0 * Omega[t] + np.dot(I[:t], Omega[t-1::-1])
    #range 25-T-1
    for t in range(25, T):
        Lambda[t] = np.dot(I[t-25:t], Omega[::-1])

    EstR = np.full(T, np.nan)
    EstR[0] = max(1, I[0] / (I_0 * Omega[0]))
    for t in range(1, tau_0):
        EstR[t] = max(1, I[t] / (I_0 * Omega[t] + np.dot(I[:t], Omega[t-1::-1])))